# K-means 1D - Etapa 1: Paralelização com OpenMP

Este notebook implementa a **Etapa 1** do projeto, focada na paralelização do K-means 1D usando OpenMP para CPUs de memória compartilhada.

O fluxo de trabalho é o seguinte:
1.  **Configuração:** Preparar os dados de entrada (`dados.csv`) e os centróides iniciais (`centroides_iniciais.csv`).
2.  **Implementação (Opção A: Reduction):** Criar a primeira versão paralela (`kmeans_1d_naive_parallel.c`) usando a cláusula `reduction` para paralelizar os laços `assignment_step` e `update_step`.
3.  **Benchmark (Opção A):** Executar um script de shell para testar a implementação A com diferentes números de threads (2, 4, 8, 16) e políticas de `schedule` (`static`, `dynamic` com vários *chunks*). O script calcula automaticamente o *Speedup* e a *Eficiência* em relação à execução com 1 thread.
4.  **Implementação (Opção B: Critical):** Criar uma segunda versão (`kmeans_1d_naive_parallel_critical.c`) que usa `#pragma omp critical` no `update_step`, como alternativa ao `reduction`.
5.  **Benchmark (Opção B):** Executar um script de benchmark similar para a implementação B, permitindo uma análise comparativa do impacto da seção crítica no desempenho.
6.  **Análise de Convergência:** Plotar os gráficos de SSE por iteração de todas as execuções para validar a corretude (garantir que todos os resultados são idênticos).

# A: Configuração dos Dados e Centróides

Esta seção prepara os arquivos de entrada necessários para todas as execuções.

1.  `dados.csv`: Gerado a partir do `city_temperature.csv`, contém ~2 milhões de registros de temperatura limpos.
2.  `centroides_iniciais.csv`: Contém 16 centróides iniciais fixos para garantir a reprodutibilidade dos testes.

In [ ]:
import pandas as pd

# Carrega o CSV, limpa os dados (-99) e salva os dados 1D (AvgTemperature)
df = pd.read_csv('/content/city_temperature.csv', dtype={'State': 'str', 'AvgTemperature': 'float'})
df = df['AvgTemperature'].replace(-99, pd.NA).dropna()
df.to_csv('dados.csv', index=False, header=False)

In [ ]:
%%writefile centroides_iniciais.csv
50.500000
88.300000
41.500000
36.400000
15.800000
79.200000
86.100000
77.200000
85.200000
84.200000
71.200000
77.300000
83.300000
61.100000
53.200000
36.600000

# B: Implementação Paralela (Opção A: Reduction)

Esta é a implementação paralela principal (`kmeans_1d_naive_parallel.c`).

- **`assignment_step_1d`**: O laço `for` principal (sobre N) é paralelizado com `#pragma omp parallel for` e usa `reduction(+:sse)` para acumular o SSE de forma segura.
- **`update_step_1d`**: O laço de acumulação (sobre N) é paralelizado usando `reduction(+:sum[:K], cnt[:K])` para acumular as somas e contagens de cada cluster de forma eficiente.
- **Medição de Tempo**: O `main` foi modificado para usar `omp_get_wtime()` para uma medição precisa do *wall-clock time*.
- **Agendamento**: `schedule(runtime)` é usado para que a política de agendamento possa ser controlada por variáveis de ambiente (`OMP_SCHEDULE`).

In [ ]:
%%writefile kmeans_1d_naive_parallel.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <omp.h>

/* ---------- util CSV 1D: (funções inalteradas) ---------- */
static int count_rows(const char *path){
    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); exit(1); }
    int rows=0; char line[8192];
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(!only_ws) rows++;
    }
    fclose(f);
    return rows;
}

static double *read_csv_1col(const char *path, int *n_out){
    int R = count_rows(path);
    if(R<=0){ fprintf(stderr,"Arquivo vazio: %s\n", path); exit(1); }
    double *A = (double*)malloc((size_t)R * sizeof(double));
    if(!A){ fprintf(stderr,"Sem memoria para %d linhas\n", R); exit(1); }
    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); free(A); exit(1); }
    char line[8192];
    int r=0;
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(only_ws) continue;
        const char *delim = ",; \t";
        char *tok = strtok(line, delim);
        if(!tok){ fprintf(stderr,"Linha %d sem valor em %s\n", r+1, path); free(A); fclose(f); exit(1); }
        A[r] = atof(tok);
        r++;
        if(r>R) break;
    }
    fclose(f);
    *n_out = R;
    return A;
}

static void write_assign_csv(const char *path, const int *assign, int N){
    // ... (código inalterado)
}

static void write_centroids_csv(const char *path, const double *C, int K){
    // ... (código inalterado)
}

/* ---------- k-means 1D: Paralelizado com OpenMP ---------- */
static double assignment_step_1d(const double *X, const double *C, int *assign, int N, int K){
    double sse = 0.0;
    #pragma omp parallel for schedule(runtime) reduction(+:sse)
    for(int i=0;i<N;i++){
        int best = -1;
        double bestd = 1e300;
        for(int c=0;c<K;c++){
            double diff = X[i] - C[c];
            double d = diff*diff;
            if(d < bestd){ bestd = d; best = c; }
        }
        assign[i] = best;
        sse += bestd;
    }
    return sse;
}

static void update_step_1d(const double *X, double *C, const int *assign, int N, int K){
    double *sum = (double*)calloc((size_t)K, sizeof(double));
    int *cnt = (int*)calloc((size_t)K, sizeof(int));
    if(!sum || !cnt){ fprintf(stderr,"Sem memoria no update\n"); exit(1); }

    // Paralelizado com reduction para os vetores sum e cnt
    #pragma omp parallel for schedule(runtime) reduction(+:sum[0:K]) reduction(+:cnt[0:K])
    for(int i = 0; i < N; i++){
        int a = assign[i];
        sum[a] += X[i];
        cnt[a] += 1;
    }

    // Laço final (rápido) permanece sequencial
    for(int c=0;c<K;c++){
        if(cnt[c] > 0) C[c] = sum[c] / (double)cnt[c];
        else           C[c] = X[0];
    }
    free(sum); free(cnt);
}

/* ---------- K-means principal ---------- */
static void kmeans_1d(const double *X, double *C, int *assign,
                      int N, int K, int max_iter, double eps,
                      int *iters_out, double *sse_out, FILE *f_sse)
{
    double prev_sse = 1e300;
    double sse = 0.0;
    int it;

    for(it=0; it<max_iter; it++){
        sse = assignment_step_1d(X, C, assign, N, K);
        if(f_sse) fprintf(f_sse, "%d,%.6f\n", it + 1, sse);

        double rel = fabs(sse - prev_sse) / (prev_sse > 0.0 ? prev_sse : 1.0);
        printf("Iteration %d, SSE: %f\n", it + 1, sse);
        if(rel < eps){ it++; break; }

        update_step_1d(X, C, assign, N, K);
        prev_sse = sse;
    }
    *iters_out = it;
    *sse_out = sse;
}

/* ---------- main ---------- */
int main(int argc, char **argv){
    if(argc < 3){
        printf("Uso: %s dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [sse_iter.csv]\n", argv[0]);
        return 1;
    }

    const char *pathX = argv[1];
    const char *pathC = argv[2];
    int max_iter = (argc>3)? atoi(argv[3]) : 50;
    double eps   = (argc>4)? atof(argv[4]) : 1e-4;
    const char *sse_file    = (argc>5)? argv[5] : NULL;

    FILE *f_sse = NULL;
    if(sse_file){
        f_sse = fopen(sse_file, "w");
        if(!f_sse){ fprintf(stderr,"Erro ao abrir %s para escrita\n", sse_file); }
        else fprintf(f_sse, "iteracao,sse\n");
    }

    int N=0, K=0;
    double *X = read_csv_1col(pathX, &N);
    double *C = read_csv_1col(pathC, &K);
    int *assign = (int*)malloc((size_t)N * sizeof(int));
    if(!assign){ fprintf(stderr,"Sem memoria para assign\n"); free(X); free(C); return 1; }

    // Medição de tempo com omp_get_wtime() para wall-clock time
    double t0 = omp_get_wtime();

    int iters = 0; double sse = 0.0;
    kmeans_1d(X, C, assign, N, K, max_iter, eps, &iters, &sse, f_sse);

    double t1 = omp_get_wtime();
    double ms = (t1 - t0) * 1000.0;

    if(f_sse) fclose(f_sse);

    printf("K-means 1D (Parallel - Reduction)\n");
    printf("N=%d K=%d max_iter=%d eps=%g\n", N, K, max_iter, eps);
    printf("Threads: %d\n", omp_get_max_threads());
    printf("Iterações: %d | SSE final: %.6f | Tempo: %.1f ms\n", iters, sse, ms);

    free(assign); free(X); free(C);
    return 0;
}


#  C: Benchmark (Opção A: Reduction)

Este script de shell automatiza o benchmark da implementação com `reduction`.

1.  Compila o código com a flag `-fopenmp`.
2.  Executa uma vez com `OMP_NUM_THREADS=1` para obter o tempo sequencial (`T_seq`) como baseline.
3.  Itera sobre o número de threads `T = {2, 4, 8, 16}`.
4.  Para cada `T`, testa `schedule(static)`.
5.  Para cada `T`, testa `schedule(dynamic)` com `chunk = {1, 10, 100, 1000}`.
6.  Calcula `Speedup = T_seq / T_paralelo` e `Eficiência = Speedup / T`.
7.  Salva todos os resultados em `resultados.csv`.

In [ ]:
%%shell
echo "Compilando a versão paralela (Reduction)..."
gcc -O2 -std=c99 -fopenmp kmeans_1d_naive_parallel.c -o kmeans_parallel_reduction -lm

# Arquivo de resultados
echo "Num_Threads,Schedule,Chunk,Tempo(ms),SSE_final,Iteracoes,Speedup,Eficiência" > resultados.csv

# Executa versão sequencial (T=1) para referência
echo "Executando o Baseline (Threads=1)..."
export OMP_NUM_THREADS=1
export OMP_SCHEDULE="static"
T_seq=$(./kmeans_parallel_reduction dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_seq.csv | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
echo "Tempo sequencial = $T_seq ms"
echo "1,static,N/A,$T_seq,N/A,N/A,1.0000,1.0000" >> resultados.csv

# Loop pelos schedules e threads
for num_t in 2 4 8 16; do
  export OMP_NUM_THREADS=$num_t

  for sched in static dynamic; do
    if [[ "$sched" == "static" ]]; then
      export OMP_SCHEDULE="static"
      chunk="padrão"

      echo "=========================================="
      echo "Executando (Reduction): $sched (chunk=$chunk, threads=$num_t)"
      echo "=========================================="

      saida=$(./kmeans_parallel_reduction dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_${num_t}_${sched}_${chunk}.csv)
      echo "$saida" | grep "Iteration"

      tempo=$(echo "$saida" | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
      sse_final=$(echo "$saida" | grep "SSE final" | sed -E 's/.*SSE final: ([0-9.eE+-]+) \|.*/\1/')
      iters=$(echo "$saida" | grep "Iterações" | sed -E 's/.*Iterações: ([0-9]+).*/\1/')
      if [[ -z "$tempo" ]]; then tempo=9999999; fi

      speedup=$(awk -v t_seq="$T_seq" -v t_par="$tempo" 'BEGIN{printf "%.4f", t_seq/t_par}')
      eficiencia=$(awk -v s="$speedup" -v n="$num_t" 'BEGIN{printf "%.4f", s/n}')

      echo "→ [$sched,$chunk] Iterações: $iters | Tempo: ${tempo}ms | SSE final: $sse_final | SpeedUp: $speedup | Eficiência: $eficiencia"
      echo "$num_t,$sched,$chunk,$tempo,$sse_final,$iters,$speedup,$eficiencia" >> resultados.csv

    else
      for chunk in 1 10 100 1000; do
        export OMP_SCHEDULE="$sched,$chunk"
        echo "=========================================="
        echo "Executando (Reduction): $sched (chunk=$chunk, threads=$num_t)"
        echo "=========================================="

        saida=$(./kmeans_parallel_reduction dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_${num_t}_${sched}_${chunk}.csv)
        echo "$saida" | grep "Iteration"

        tempo=$(echo "$saida" | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
        sse_final=$(echo "$saida" | grep "SSE final" | sed -E 's/.*SSE final: ([0-9.eE+-]+) \|.*/\1/')
        iters=$(echo "$saida" | grep "Iterações" | sed -E 's/.*Iterações: ([0-9]+).*/\1/')
        if [[ -z "$tempo" ]]; then tempo=9999999; fi

        speedup=$(awk -v t_seq="$T_seq" -v t_par="$tempo" 'BEGIN{printf "%.4f", t_seq/t_par}')
        eficiencia=$(awk -v s="$speedup" -v n="$num_t" 'BEGIN{printf "%.4f", s/n}')

        echo "→ [$sched,$chunk] Iterações: $iters | Tempo: ${tempo}ms | SSE final: $sse_final | SpeedUp: $speedup | Eficiência: $eficiencia"
        echo "$num_t,$sched,$chunk,$tempo,$sse_final,$iters,$speedup,$eficiencia" >> resultados.csv
      done
    fi
  done
done

echo -e "\n=========== RESULTADOS FINAIS (REDUCTION) ==========="
column -t -s, resultados.csv


#  D: Implementação Paralela (Opção B: Critical)

Esta é a implementação paralela alternativa (`kmeans_1d_naive_parallel_critical.c`).

- **`assignment_step_1d`**: Permanece igual, paralelizado com `reduction(+:sse)`.
- **`update_step_1d`**: O laço de acumulação é paralelizado, mas em vez de `reduction`, ele usa `#pragma omp critical` para proteger as escritas nos vetores `sum` e `cnt`.

O objetivo é comparar o desempenho desta abordagem, que serializa as atualizações (criando um gargalo), com a abordagem mais eficiente de `reduction`.

In [ ]:
%%writefile kmeans_1d_naive_parallel_critical.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <omp.h>

/* ---------- util CSV 1D: (funções inalteradas) ---------- */
static int count_rows(const char *path){ /* ... (código inalterado) ... */ }
static double *read_csv_1col(const char *path, int *n_out){ /* ... (código inalterado) ... */ }
static void write_assign_csv(const char *path, const int *assign, int N){ /* ... (código inalterado) ... */ }
static void write_centroids_csv(const char *path, const double *C, int K){ /* ... (código inalterado) ... */ }

/* ---------- k-means 1D: Paralelizado com OpenMP ---------- */
static double assignment_step_1d(const double *X, const double *C, int *assign, int N, int K){
    double sse = 0.0;
    #pragma omp parallel for schedule(runtime) reduction(+:sse)
    for(int i=0;i<N;i++){
        int best = -1;
        double bestd = 1e300;
        for(int c=0;c<K;c++){
            double diff = X[i] - C[c];
            double d = diff*diff;
            if(d < bestd){ bestd = d; best = c; }
        }
        assign[i] = best;
        sse += bestd;
    }
    return sse;
}

/* update: Versão alternativa usando omp critical */
static void update_step_1d(const double *X, double *C, const int *assign, int N, int K){
    double *sum = (double*)calloc((size_t)K, sizeof(double));
    int *cnt = (int*)calloc((size_t)K, sizeof(int));
    if(!sum || !cnt){ fprintf(stderr,"Sem memoria no update\n"); exit(1); }

    #pragma omp parallel for schedule(runtime)
    for(int i = 0; i < N; i++){
        int a = assign[i];
        // Seção crítica serializa o acesso, criando um gargalo
        #pragma omp critical
        {
            sum[a] += X[i];
            cnt[a] += 1;
        }
    }
    for(int c=0;c<K;c++){
        if(cnt[c] > 0) C[c] = sum[c] / (double)cnt[c];
        else           C[c] = X[0];
    }
    free(sum); free(cnt);
}

/* ---------- K-means principal (código inalterado) ---------- */
static void kmeans_1d(const double *X, double *C, int *assign,
                      int N, int K, int max_iter, double eps,
                      int *iters_out, double *sse_out, FILE *f_sse)
{
    double prev_sse = 1e300;
    double sse = 0.0;
    int it;
    for(it=0; it<max_iter; it++){
        sse = assignment_step_1d(X, C, assign, N, K);
        if(f_sse) fprintf(f_sse, "%d,%.6f\n", it + 1, sse);
        double rel = fabs(sse - prev_sse) / (prev_sse > 0.0 ? prev_sse : 1.0);
        printf("Iteration %d, SSE: %f\n", it + 1, sse);
        if(rel < eps){ it++; break; }
        update_step_1d(X, C, assign, N, K);
        prev_sse = sse;
    }
    *iters_out = it;
    *sse_out = sse;
}

/* ---------- main (código inalterado, exceto printf) ---------- */
int main(int argc, char **argv){
    if(argc < 3){
        printf("Uso: %s dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [sse_iter.csv]\n", argv[0]);
        return 1;
    }
    const char *pathX = argv[1];
    const char *pathC = argv[2];
    int max_iter = (argc>3)? atoi(argv[3]) : 50;
    double eps   = (argc>4)? atof(argv[4]) : 1e-4;
    const char *sse_file    = (argc>5)? argv[5] : NULL;
    FILE *f_sse = NULL;
    if(sse_file){
        f_sse = fopen(sse_file, "w");
        if(!f_sse){ fprintf(stderr,"Erro ao abrir %s para escrita\n", sse_file); }
        else fprintf(f_sse, "iteracao,sse\n");
    }
    int N=0, K=0;
    double *X = read_csv_1col(pathX, &N);
    double *C = read_csv_1col(pathC, &K);
    int *assign = (int*)malloc((size_t)N * sizeof(int));
    if(!assign){ fprintf(stderr,"Sem memoria para assign\n"); free(X); free(C); return 1; }
    double t0 = omp_get_wtime();
    int iters = 0; double sse = 0.0;
    kmeans_1d(X, C, assign, N, K, max_iter, eps, &iters, &sse, f_sse);
    double t1 = omp_get_wtime();
    double ms = (t1 - t0) * 1000.0;
    if(f_sse) fclose(f_sse);
    printf("K-means 1D (Parallel - Critical)\n");
    printf("N=%d K=%d max_iter=%d eps=%g\n", N, K, max_iter, eps);
    printf("Threads: %d\n", omp_get_max_threads());
    printf("Iterações: %d | SSE final: %.6f | Tempo: %.1f ms\n", iters, sse, ms);
    free(assign); free(X); free(C);
    return 0;
}


# E: Benchmark (Opção B: Critical)

Este script de shell executa o benchmark para a implementação com `#pragma omp critical`.

O processo é idêntico ao da Etapa C, mas os resultados são salvos em `resultados_critical.csv` para comparação direta.

In [ ]:
%%shell
echo "Compilando a versão paralela (Critical)..."
gcc -O2 -std=c99 -fopenmp kmeans_1d_naive_parallel_critical.c -o kmeans_parallel_critical -lm

# Arquivo de resultados
echo "Num_Threads,Schedule,Chunk,Tempo(ms),SSE_final,Iteracoes,Speedup,Eficiência" > resultados_critical.csv

# Executa versão sequencial (T=1) para referência
echo "Executando o Baseline (Threads=1)..."
export OMP_NUM_THREADS=1
export OMP_SCHEDULE="static"
T_seq=$(./kmeans_parallel_critical dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_seq_critical.csv | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
echo "Tempo sequencial = $T_seq ms"
echo "1,static,N/A,$T_seq,N/A,N/A,1.0000,1.0000" >> resultados_critical.csv

# Loop pelos schedules e threads
for num_t in 2 4 8 16; do
  export OMP_NUM_THREADS=$num_t

  for sched in static dynamic; do
    if [[ "$sched" == "static" ]]; then
      export OMP_SCHEDULE="static"
      chunk="padrão"

      echo "=========================================="
      echo "Executando (Critical): $sched (chunk=$chunk, threads=$num_t)"
      echo "=========================================="

      saida=$(./kmeans_parallel_critical dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_${num_t}_${sched}_${chunk}_critical.csv)
      echo "$saida" | grep "Iteration"

      tempo=$(echo "$saida" | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
      sse_final=$(echo "$saida" | grep "SSE final" | sed -E 's/.*SSE final: ([0-9.eE+-]+) \|.*/\1/')
      iters=$(echo "$saida" | grep "Iterações" | sed -E 's/.*Iterações: ([0-9]+).*/\1/')
      if [[ -z "$tempo" ]]; then tempo=9999999; fi

      speedup=$(awk -v t_seq="$T_seq" -v t_par="$tempo" 'BEGIN{printf "%.4f", t_seq/t_par}')
      eficiencia=$(awk -v s="$speedup" -v n="$num_t" 'BEGIN{printf "%.4f", s/n}')

      echo "→ [$sched,$chunk] Iterações: $iters | Tempo: ${tempo}ms | SSE final: $sse_final | SpeedUp: $speedup | Eficiência: $eficiencia"
      echo "$num_t,$sched,$chunk,$tempo,$sse_final,$iters,$speedup,$eficiencia" >> resultados_critical.csv

    else
      for chunk in 1 10 100 1000; do
        export OMP_SCHEDULE="$sched,$chunk"
        echo "=========================================="
        echo "Executando (Critical): $sched (chunk=$chunk, threads=$num_t)"
        echo "=========================================="

        saida=$(./kmeans_parallel_critical dados.csv centroides_iniciais.csv 500 0.0001 sse_evolution_${num_t}_${sched}_${chunk}_critical.csv)
        echo "$saida" | grep "Iteration"

        tempo=$(echo "$saida" | grep "Tempo:" | tail -n1 | sed -E 's/.*Tempo: ([0-9.]+) ms.*/\1/')
        sse_final=$(echo "$saida" | grep "SSE final" | sed -E 's/.*SSE final: ([0-9.eE+-]+) \|.*/\1/')
        iters=$(echo "$saida" | grep "Iterações" | sed -E 's/.*Iterações: ([0-9]+).*/\1/')
        if [[ -z "$tempo" ]]; then tempo=9999999; fi

        speedup=$(awk -v t_seq="$T_seq" -v t_par="$tempo" 'BEGIN{printf "%.4f", t_seq/t_par}')
        eficiencia=$(awk -v s="$speedup" -v n="$num_t" 'BEGIN{printf "%.4f", s/n}')

        echo "→ [$sched,$chunk] Iterações: $iters | Tempo: ${tempo}ms | SSE final: $sse_final | SpeedUp: $eficiencia"
        echo "$num_t,$sched,$chunk,$tempo,$sse_final,$iters,$speedup,$eficiencia" >> resultados_critical.csv
      done
    fi
  done
done

echo -e "\n=========== RESULTADOS FINAIS (CRITICAL) ==========="
column -t -s, resultados_critical.csv


#  F: Análise de Convergência (Validação)

Finalmente, plotamos os arquivos `sse_evolution_*.csv` gerados por todas as execuções. O objetivo é validar a corretude: todos os gráficos de convergência do SSE devem ser idênticos, provando que, apesar das diferentes técnicas de paralelização e tempos de execução, o resultado numérico final foi o mesmo.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Itera sobre todos os arquivos de log de SSE gerados
for file in os.listdir('/content'):
    if file.startswith('sse_evolution_') and file.endswith('.csv'):
        path = os.path.join('/content', file)
        try:
            # Tenta ler o CSV, lidando com ou sem cabeçalho
            try:
                sse_df = pd.read_csv(path)
                if not {'iteracao', 'sse'}.issubset(sse_df.columns):
                    sse_df.columns = ['iteracao', 'sse']
            except Exception:
                sse_df = pd.read_csv(path, header=None, names=['iteracao', 'sse'])

            # Limpa dados não numéricos que podem ter sido mal interpretados
            sse_df = sse_df[pd.to_numeric(sse_df['iteracao'], errors='coerce').notna()]
            sse_df = sse_df[pd.to_numeric(sse_df['sse'], errors='coerce').notna()]

            # Converte os tipos para garantir a plotagem correta
            sse_df['iteracao'] = sse_df['iteracao'].astype(int)
            sse_df['sse'] = sse_df['sse'].astype(float)

            if not sse_df.empty:
                plt.figure(figsize=(8, 5))
                plt.plot(sse_df['iteracao'], sse_df['sse'], linewidth=1.5)
                plt.xlabel('Iteração')
                plt.ylabel('SSE (Sum of Squared Errors)')
                plt.title(f'Convergência ({file.replace("sse_evolution_", "").replace(".csv","")})')
                plt.grid(True)
                plt.tight_layout()
                plt.show()
        except Exception as e:
            print(f"Erro ao processar o arquivo {file}: {e}")
